In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
!pip install evaluate transformers accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00


In [ ]:
import evaluate

In [ ]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, roc_auc_score, roc_curve, auc,
                             classification_report)

import transformers
import accelerate
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoTokenizer
from transformers import Trainer
from transformers import TrainingArguments
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    default_data_collator
)

import datasets
from datasets import load_dataset
from datasets import load_dataset_builder
from datasets import Dataset
from datasets import load_metric

In [ ]:
transformers.__version__, accelerate.__version__

('4.35.2', '0.25.0')

In [ ]:
DATASET_PATHS = {
 #   'train': 'drive/MyDrive/Xackaton/train.json',
    'train': 'drive/MyDrive/Xackaton/train.csv',
    'test': 'drive/MyDrive/Xackaton/val.csv',
    'sample_sub': 'drive/MyDrive/Xackaton/test.csv'
}
OUTPUT_DICT = 'drive/MyDrive/Xackaton/'
LABEL_TO_ID = {'female': 0, 'male': 1}
ID_TO_LABEL = {v:k for k, v in LABEL_TO_ID.items()}

### Dataset prep

In [ ]:
# Set Dataset
#train_d = pd.read_json(DATASET_PATHS['train']).dropna()
train_df = pd.read_csv(DATASET_PATHS['train'], sep=',').dropna()
# train_df.drop(train_df.columns[0], axis=1, inplace=True)  # remove id column
#train_df = train_d.T
train_df.rename(columns={'features': 'text', 'target': 'label'}, inplace=True)  # columns to lower case
train_df.replace({'label': LABEL_TO_ID}, inplace=True)


### Dataset prep

In [ ]:
train_df.head()

,label,text
0,0,orders site id 1 orders site id 2 orders site...
1,0,visits site id 3 visits session duration 0 pa...
2,1,orders site id 21 orders site id 22 orders it...
3,1,orders site id 2 orders items id item 8 count...
4,0,orders site id 39 orders site id 40 orders it...


In [ ]:
# Create Dataset
train_dataset = Dataset.from_pandas(train_df)
#train_dataset = train_dataset.remove_columns(list(train_dataset.features.keys())[-1])  # remove id last column
train_dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 127755
})

### Model prep

In [ ]:
MODEL_NAME = "distilbert-base-uncased"
TOKENIZER_CHEKPOINT = "distilbert-base-uncased"
#TOKENIZER_CHEKPOINT = "greatakela/bertson"

#MODEL_NAME = "bert-base-uncased"
#TOKENIZER_CHEKPOINT = "bert-base-uncased"
LABEL_NUM = 2
MAX_LENGTH = 512

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_CHEKPOINT)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(x):
    return tokenizer(x['text'], padding=True, truncation=True, max_length=512
    )

In [ ]:
'''def tokenize_function(x):
    return tokenizer(str(x['text']), truncation=True, max_length=1000, padding=True
    )'''

In [ ]:
# Tokenizer for Train Dataset
tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets

Map:   0%|          | 0/127755 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 127755
})

In [ ]:
len(tokenized_datasets['input_ids'][108000])

512

In [ ]:
tokenized_datasets['text'][4]

' orders site id 39 orders site id 40 orders items id item 13 count 1 general category path 91491 91461 198119 brand id 1 id item 14 count 1 general category path 91072 91497 91461 198119 brand id 1161 site id 20 orders items id item 15 count 1 general category path 90508 13199877 90402 brand id 4315 id item 16 count 1 id item 17 count 1 visits site id 3 visits session duration 0 pages count 1 session duration 0 pages count 1 session duration 110 pages count 2 site id 23 visits session duration 115 pages count 2 visited items item 18 item 19 site id 13 visits session duration 0 pages count 1 session duration 582 pages count 6 session duration 0 pages count 1 session duration 0 pages count 1 session duration 0 pages count 1 session duration 1 pages count 2 session duration 381 pages count 3 site id 41 visits session duration 224 pages count 5 visited items item 20 session duration 62 pages count 7 site id 20 visits session duration 771 pages count 1 visited items item 17 item 16 item 15

In [ ]:
#Model

In [ ]:
config = AutoConfig.from_pretrained(MODEL_NAME, num_labels=LABEL_NUM)#, max_position_embeddings = MAX_LENGTH)
#tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
#tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_CHEKPOINT)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    config=config,
    ignore_mismatched_sizes=True
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#Train

In [ ]:
BATCH_SIZE_TRAIN = 16 #16
BATCH_SIZE_EVAL = 16 #32
EPOCHS_NUM = 4 #3
SPLIT_RATE = 0.15
#OPTIM = "adamw_torch"
OPTIM = "adamw_torch"

In [ ]:
'''def compute_metrics(eval_preds):
    metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)'''

In [ ]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [ ]:
model.config.label2id = LABEL_TO_ID
model.config.id2label = {id: label for label, id in config.label2id.items()}

In [ ]:
splitted_data= train_dataset.train_test_split(test_size=SPLIT_RATE)
splitted_data

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 108591
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 19164
    })
})

In [ ]:
#model

In [ ]:
tokenized_train = splitted_data['train'].map(tokenize_function, batched=True)
tokenized_val = splitted_data['test'].map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/108591 [00:00<?, ? examples/s]

Map:   0%|          | 0/19164 [00:00<?, ? examples/s]

In [ ]:
tokenized_train['text'][3]

In [ ]:
training_args = TrainingArguments(
    output_dir="drive/MyDrive/Xackaton/dbtest-trainer",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    #load_best_model_at_end=True,
    #push_to_hub=True,
    num_train_epochs=EPOCHS_NUM,
    per_device_train_batch_size=BATCH_SIZE_TRAIN,
    per_device_eval_batch_size=BATCH_SIZE_EVAL,
    warmup_steps=500,
    weight_decay=0.01,
    #logging_steps=100,
    optim=OPTIM
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
transformers.logging.set_verbosity('CRITICAL')
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.454800,0.459447,0.787727,0.780937
2,0.443500,0.443868,0.800616,0.811094
3,0.411900,0.441018,0.802546,0.805370
4,0.394200,0.454974,0.803799,0.807987


<ipython-input-19-06cfd0d0f79d>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


TrainOutput(global_step=27148, training_loss=0.436486188417263, metrics={'train_runtime': 7007.3666, 'train_samples_per_second': 61.987, 'train_steps_per_second': 3.874, 'total_flos': 5.753906914981478e+16, 'train_loss': 0.436486188417263, 'epoch': 4.0})

In [ ]:
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1702989035.a47ac9e33125.600.0:   0%|          | 0.00/14.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

'https://huggingface.co/greatakela/dbtest-trainer/tree/main/'

In [ ]:
# evaluate

In [ ]:
# Set Dataset
test_df = pd.read_csv(DATASET_PATHS['test'], sep=',').dropna()
test_df.rename(columns={'features': 'text', 'target': 'label'}, inplace=True)  # columns to lower case
test_df.replace({'label': LABEL_TO_ID}, inplace=True)

In [ ]:
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/27447 [00:00<?, ? examples/s]

In [ ]:
preds_output = trainer.predict(tokenized_test_datasets)
y_preds = np.argmax(preds_output.predictions,axis=1)

In [ ]:
y_true = tokenized_test_datasets['label']

In [ ]:
print(classification_report(y_true, y_preds, target_names = ['0', '1']))

              precision    recall  f1-score   support

           0       0.81      0.78      0.80     13726
           1       0.79      0.82      0.80     13721

    accuracy                           0.80     27447
   macro avg       0.80      0.80      0.80     27447
weighted avg       0.80      0.80      0.80     27447



In [ ]:
test = pd.read_csv(DATASET_PATHS['sample_sub'], sep=',').dropna()
test

,Unnamed: 0,features
0,user_155203,visits site id 20 visits session duration 130...
1,user_155204,orders site id 489 orders visits site id 161 ...
2,user_155205,visits site id 3 visits session duration 0 pa...
3,user_155206,orders site id 62 orders items id item 13288 ...
4,user_155207,orders site id 104 orders items id item 18901...
...,...,...
18779,user_173982,orders site id 21 orders site id 29 orders it...
18780,user_173983,orders site id 34 orders items id item 311107...
18781,user_173984,orders site id 29 orders items id item 584480...
18782,user_173985,orders site id 114 orders items id item 20422...


In [ ]:
test.rename(columns={'features': 'text'}, inplace=True)  # columns to lower case
test.replace({'label': LABEL_TO_ID}, inplace=True)

In [ ]:
def get_prediction(text):
    inputs = tokenizer(text, truncation=True,padding=True, return_tensors="pt").to("cuda")
    outputs = model(**inputs)
    probs = outputs[0].softmax(1)
    return model.config.id2label[probs.argmax().item()]

In [ ]:
tokenized_test = test_dataset.map(tokenize_function, batched=True)
trainer.evaluate(tokenized_test)

In [ ]:
test['target'] = test['text'].apply(lambda text: get_prediction(text))
test.head()

,Unnamed: 0,text,target
0,user_155203,visits site id 20 visits session duration 130...,male
1,user_155204,orders site id 489 orders visits site id 161 ...,male
2,user_155205,visits site id 3 visits session duration 0 pa...,male
3,user_155206,orders site id 62 orders items id item 13288 ...,female
4,user_155207,orders site id 104 orders items id item 18901...,female


In [ ]:
#submission = test[['id', 'Sentiment']]
submission = test.copy()
submission.to_csv(('drive/MyDrive/Xackaton/submissionDB.csv'), index=False)

In [ ]:
test.rename(columns={'Unnamed: 0': 'user_id'}, inplace=True)  # columns to lower case

In [ ]:
submit = test[['user_id', 'target']]
submit.to_csv(('drive/MyDrive/Xackaton/submitDB.csv'), index=False)

In [ ]:
submit

,user_id,target
0,user_155203,male
1,user_155204,male
2,user_155205,male
3,user_155206,female
4,user_155207,female
...,...,...
18779,user_173982,male
18780,user_173983,male
18781,user_173984,female
18782,user_173985,female
